In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
import os
import datetime

from pytz import timezone, utc
import pandas as pd
import numpy as np

def to_NZDT(dates):
    nzdt = []
    utcs = []
    nztz = timezone('Pacific/Auckland')
    
    for _dt in dates:
        date_utc = utc.localize(_dt.to_pydatetime())
        date_nzdt = date_utc.astimezone(nztz)
        nzdt.append(str(date_nzdt))
        utcs.append(str(date_utc))
    return pd.DataFrame({'date_nzdt': nzdt, 'date_utc':utcs})

def FITS_download(base_url="https://fits.geonet.org.nz/observation", new=True):
    """
    Download all gas flux data for White Island.
    """
    url = "{}?siteID={}&typeID=SO2-flux-a&methodID={}"
    dfs = []
    for station in ['WI301', 'WI302']:
        for method in ['mdoas-ah', 'mdoas-ch']:
            fits_cache = os.path.join('/tmp', "Plot_{}_{}.csv".format(station, method))
            if os.path.isfile(fits_cache) and not new:
                df = pd.read_csv(fits_cache, parse_dates=[0],
                             skiprows=1, names=[method.replace('mdoas-','')])
            else:
                df = pd.read_csv(url.format(base_url, station, method), 
                                 parse_dates=[0],
                                 skiprows=1, names=['date', method.replace('mdoas-','')],
                                 usecols=[0, 1])
                df.to_csv(fits_cache)
            dfs.append(df)
    return dfs
dfs = FITS_download(new=True)

In [ ]:
df1 = pd.merge(dfs[0], dfs[1], left_on='date', right_on='date', how='outer')
df2 = pd.merge(dfs[2], dfs[3], left_on='date', right_on='date', how='outer')
index1 = pd.MultiIndex.from_arrays([['WI301', 'WI301', 'WI301'],['date', 'ah', 'ch']])
df1.columns = index1
index2 = pd.MultiIndex.from_arrays([['WI302', 'WI302', 'WI302'],['date', 'ah', 'ch']])
df2.columns = index2
df_nzdt = to_NZDT(df1['WI301']['date'])
cols = pd.MultiIndex.from_arrays([['WI301', 'WI301'],['NZDT', 'UTC']])
df_nzdt.columns = cols
df1 = pd.merge(df1, df_nzdt, left_index=True, right_index=True, how='outer')
#df_tmp = pd.DataFrame({'ah-tpd': df1.loc[:, ('WI301','mdoas-ah')].values*86.4})
#df_tmp.columns = pd.MultiIndex.from_arrays([['WI301'],['ah-tpd']])
#df1 = pd.merge(df1, df_tmp, left_index=True, right_index=True, how='outer')
df1.loc[:, ('WI301','ahtpd')] = df1.loc[:, ('WI301','ah')]*86.4
df1.loc[:, ('WI301','chtpd')] = df1.loc[:, ('WI301','ch')]*86.4

df_nzdt = to_NZDT(df2['WI302']['date'])
cols = pd.MultiIndex.from_arrays([['WI302', 'WI302'],['NZDT', 'UTC']])
df_nzdt.columns = cols
df2 = pd.merge(df2, df_nzdt, left_index=True, right_index=True, how='outer')
df2.loc[:, ('WI302','ahtpd')] = df2.loc[:, ('WI302','ah')]*86.4
df2.loc[:, ('WI302','chtpd')] = df2.loc[:, ('WI302','ch')]*86.4

In [ ]:
from bokeh.models import HoverTool
ah_hover = HoverTool(tooltips=[
    ("ah [kg/s]", "@ah"),
    ("ah [t/d]", "@ahtpd"),
    ("date", "@UTC"),
    ("NZDT", "@NZDT"),
])



ch_hover = HoverTool(tooltips=[
    ("ch [kg/s]", "@ch"),
    ("ch [t/d]", "@chtpd"),
    ("date", "@UTC"),
    ("NZDT", "@NZDT"),
])

In [ ]:
%%output size=150
%%opts  Scatter [height=200 width=600] (size=5)
scatter_dict = {'mdoas-ah': hv.Scatter(df1['WI301'], kdims=['date'], vdims=['ah', 'NZDT', 
                                                                            'UTC', 'ahtpd']).opts(plot=dict(tools=[ah_hover])),
                'mdoas-ch': hv.Scatter(df1['WI301'], kdims=['date'], vdims=['ch', 'NZDT', 
                                                                            'UTC', 'chtpd']).opts(plot=dict(tools=[ch_hover]))}
nd = hv.NdOverlay(scatter_dict, kdims='type')
nd


In [ ]:
%%output size=150
%%opts  Scatter [height=200 width=600] (size=5)
scatter_dict = {'mdoas-ah': hv.Scatter(df2['WI302'], kdims=['date'], vdims=['ah', 'NZDT', 
                                                                            'UTC', 'ahtpd']).opts(plot=dict(tools=[ah_hover])),
                'mdoas-ch': hv.Scatter(df2['WI302'], kdims=['date'], vdims=['ch', 'NZDT', 
                                                                            'UTC', 'chtpd']).opts(plot=dict(tools=[ch_hover]))}
nd = hv.NdOverlay(scatter_dict, kdims='type')
nd
